why is metakb not in the requirements?

In [2]:
import pandas as pd
import os
import json
import pronto

In [8]:
annotated_poda_df = pd.read_csv(
    "../../../../Pediatric dataset annotations.csv", sep=","
)
annotated_poda_df = annotated_poda_df.loc[:, ~annotated_poda_df.columns.str.startswith("Unnamed")]
annotated_poda_df

,FDA Brand Label,INDICATION 2,PEDIATRIC APPROVAL DATE 3*,Variation,GENE_ID,GENETIC_LABELS,DISEASE_LABELS,DISEASE_ID
0,Asparaginase [Elspar],As a component of a multi-agent chemotherapeut...,01-10-1978,--,NaN,--,acute lymphoblastic leukemia (ALL),MONDO:0004967
1,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,hypersensitivity to E. coli-derived asparagina...,MONDO:0005271
2,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,lymphoblastic lymphoma (LBL),MONDO:0000873
3,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,acute lymphoblastic leukemia (ALL),MONDO:0004967
4,Asparaginase Erwinia Chrysanthemi [Erwinaze] [...,A component of a multi-agent chemotherapeutic ...,11-18-2011,--,NaN,--,acute lymphoblastic leukemia (ALL),MONDO:0004967
...,...,...,...,...,...,...,...,...
105,Trametinib [Mekinist],"In combination with dabrafenib, for the treatm...",8-31-2023 *,V600E mutation,HGNC:1097,BRAF,metastatic neoplasm,MONDO:0024883
106,Trametinib [Mekinist],"In combination with dabrafenib, for the treatm...",03-16-2023,V600E mutation,HGNC:1097,BRAF,Low Grade Glioma,MONDO:0021637
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
key_map = {
    "INDICATION 2": "description",
    "GENETIC_LABELS": "geneContextQualifier",
    "FDA Brand Label": "objectTherapeutic"
}
multi_value_column = "GENETIC_LABELS"

prop_type = "VariantTherapeuticResponseProposition"
predicate = "sensitivity"

output_dir = "../../../../json_outputs"

for idx, row in annotated_poda_df.iterrows():
    values = [v.strip() for v in str(row[multi_value_column]).split(",")]

    for i, val in enumerate(values):
        json_obj = {"type": prop_type, "predicate": predicate}

        for col in key_map:
            key = key_map[col]
            value = val if col == multi_value_column else row[col]
            json_obj[key] = value

        filename = f"row_{idx}_{i}.json"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w") as f:
            json.dump(json_obj, f, indent=4)


earlier the link was working which is preferable but now it is the local copy of mondo.obo

In [10]:
# Load the OBO file
ontology = pronto.Ontology("mondo.obo")

/var/folders/vt/znzp_c6s02q6kjcmqfk0cb500000gq/T/ipykernel_18486/3556618749.py:2: UnicodeWarning: unsound encoding, assuming utf-8 (99% confidence)
  ontology = pronto.Ontology("mondo.obo")


In [42]:
cleaned_annotated_poda_df = annotated_poda_df.copy()

In [43]:
cleaned_annotated_poda_df['DISEASE_LABELS'] = cleaned_annotated_poda_df['DISEASE_LABELS'].str.replace(r'\s*\([^)]*\)\s*', '', regex=True)


In [44]:
search_terms = cleaned_annotated_poda_df["DISEASE_LABELS"].dropna()

In [45]:
results = []

for idx, search_string in search_terms.items():
    if search_string == "--":
        results.append({
            "index": idx,
            "method_ID": None,
            "method_name": None,
            "method_match_type": "skipped"
        })
        continue

    exact_match = None
    match_candidates = []

    # 1. Exact match on term name
    for term in ontology.terms():
        if (
            term.id and term.id.startswith("MONDO") and
            term.name and term.name.lower() == search_string.lower()
        ):
            exact_match = term
            break

    if exact_match:
        results.append({
            "index": idx,
            "method_ID": exact_match.id,
            "method_name": exact_match.name,
            "method_match_type": "exact match"
        })
        continue  # skip synonym matching

    # 2. Synonym match
    for term in ontology.terms():
        if term.id and term.id.startswith("MONDO") and term.synonyms:
            for syn in term.synonyms:
                if syn.description.lower() == search_string.lower():
                    scope = syn.scope.upper() if syn.scope else ''
                    rank = {'EXACT': 2, 'RELATED': 1}.get(scope, 0)
                    match_candidates.append((rank, term, scope))
                    break  # only first matching synonym per term

    if match_candidates:
        match_candidates.sort(reverse=True, key=lambda x: x[0])
        top_rank = match_candidates[0][0]
        top_matches = [item for item in match_candidates if item[0] == top_rank]

        # We'll take the first top match for consistency
        _, top_term, scope = top_matches[0]
        match_type = (
            "exact synonym match" if top_rank == 2
            else "other synonym match"
        )

        results.append({
            "index": idx,
            "method_ID": top_term.id,
            "method_name": top_term.name,
            "method_match_type": match_type
        })
    else:
        results.append({
            "index": idx,
            "method_ID": None,
            "method_name": None,
            "method_match_type": "no match"
        })

# Convert results list to DataFrame
match_df = pd.DataFrame(results).set_index("index")

# Merge match results back into a copy of the original DataFrame
annotated_with_matches_df = cleaned_annotated_poda_df.copy()
annotated_with_matches_df = annotated_with_matches_df.join(match_df)


In [46]:
annotated_with_matches_df

,FDA Brand Label,INDICATION 2,PEDIATRIC APPROVAL DATE 3*,Variation,GENE_ID,GENETIC_LABELS,DISEASE_LABELS,DISEASE_ID,method_ID,method_name,method_match_type
0,Asparaginase [Elspar],As a component of a multi-agent chemotherapeut...,01-10-1978,--,NaN,--,acute lymphoblastic leukemia,MONDO:0004967,MONDO:0004967,acute lymphoblastic leukemia,exact match
1,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,hypersensitivity to E. coli-derived asparagina...,MONDO:0005271,None,None,no match
2,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,lymphoblastic lymphoma,MONDO:0000873,MONDO:0000873,lymphoblastic lymphoma,exact match
3,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,acute lymphoblastic leukemia,MONDO:0004967,MONDO:0004967,acute lymphoblastic leukemia,exact match
4,Asparaginase Erwinia Chrysanthemi [Erwinaze] [...,A component of a multi-agent chemotherapeutic ...,11-18-2011,--,NaN,--,acute lymphoblastic leukemia,MONDO:0004967,MONDO:0004967,acute lymphoblastic leukemia,exact match
...,...,...,...,...,...,...,...,...,...,...,...
105,Trametinib [Mekinist],"In combination with dabrafenib, for the treatm...",8-31-2023 *,V600E mutation,HGNC:1097,BRAF,metastatic neoplasm,MONDO:0024883,MONDO:0024883,metastatic neoplasm,exact match
106,Trametinib [Mekinist],"In combination with dabrafenib, for the treatm...",03-16-2023,V600E mutation,HGNC:1097,BRAF,Low Grade Glioma,MONDO:0021637,MONDO:0021637,low grade glioma,exact match
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


in all the cases below, the method annotation is more accurate.

In [49]:
mismatched_rows = annotated_with_matches_df[
    (annotated_with_matches_df["DISEASE_ID"] != annotated_with_matches_df["method_ID"]) &
    (annotated_with_matches_df["method_ID"].notna())
]
mismatched_rows


,FDA Brand Label,INDICATION 2,PEDIATRIC APPROVAL DATE 3*,Variation,GENE_ID,GENETIC_LABELS,DISEASE_LABELS,DISEASE_ID,method_ID,method_name,method_match_type
29,Iobenguane I-131 [Azedra],Treatment of adult and pediatric patients 12 y...,07-30-2018,--,NaN,--,pheochromocytoma-paraganglioma,MONDO:0017366,MONDO:0035540,pheochromocytoma-paraganglioma,exact match
100,Arsenic trioxide [Trisenox],Induction of remission and consolidation in pa...,09-25-2000,translocation,HGNC:9113 | HGNC:9864,PML | RARA,Acute Promyelocytic Leukemia with PML-RARA,MONDO:0100375,MONDO:0012883,acute promyelocytic leukemia,exact synonym match
101,Revumenib [Revuforj],Treatment of relapsed or refractory acute leuk...,11-15-2024,Translocation,HGNC:7132,KMT2A,Acute Undifferentiated Leukemia,MONDO:0010643,MONDO:0020321,acute undifferentiated leukemia,exact match


In [51]:
unannotated_method_rows = annotated_with_matches_df[
    (annotated_with_matches_df["method_ID"].isna())
]
unannotated_method_rows

,FDA Brand Label,INDICATION 2,PEDIATRIC APPROVAL DATE 3*,Variation,GENE_ID,GENETIC_LABELS,DISEASE_LABELS,DISEASE_ID,method_ID,method_name,method_match_type
1,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,hypersensitivity to E. coli-derived asparagina...,MONDO:0005271,None,None,no match
5,Asparaginase Erwinia Chrysanthemi [Erwinaze] [...,A component of a multi-agent chemotherapeutic ...,11-18-2011,--,NaN,--,hypersensitivity to E. coli-derived asparagina...,MONDO:0005271,None,None,no match
14,Cyclophosphamide [Cytoxan],MULTIPLE,11-16-1959,--,NaN,--,--,NaN,None,None,skipped
20,Daunorubicin and cytarabine liposome [Vyxeos],Treatment of newly-diagnosed therapy-related a...,03-30-2021,--,NaN,--,therapy related acute myeloid leukemia/AML wit...,MONDO:0019457,None,None,no match
24,Doxorubicin Hydrochloride,MULTIPLE,08-07-1974,--,NaN,--,--,NaN,None,None,skipped
32,Larotrectinib [Vitrakvi],Treatment of adult and pediatric patients with...,11-26-2018,--,NaN,--,solid tumors that have a neurotrophic receptor...,MONDO:0700215,None,None,no match
35,Lutetium Lu 177 dotatate [Lutathera],Treatment of adult and pediatric patients 12 y...,04-23-2024,--,NaN,--,gastroenteropancreatic neuroendocrine tumors,MONDO:0024503,None,None,no match
39,Methotrexate,Treatment of adults and pediatric patients wit...,12-07-1953,--,NaN,--,Non-hodgkin Lymphoma,MONDO:0018908,None,None,no match
40,Methotrexate,Treatment of adults and pediatric patients wit...,12-07-1953,--,NaN,--,Osteosarcoma,MONDO:0009807,None,None,no match
42,Nelarabine [Arranon],Treatment of T-cell acute lymphoblastic leukem...,10-28-2005,--,NaN,--,T-cell lymphoblastic lymphoma,MONDO:0044917,None,None,no match


In [53]:
unannotated_method_rows["DISEASE_LABELS"].value_counts()

DISEASE_LABELS
Philadelphia chromosome positive chronic myeloid leukemia                         5
B-cell precursor acute lymphoblastic leukemia                                     3
--                                                                                2
Acute Lymphoblastic Leukemia                                                      2
hypersensitivity to E. coli-derived asparaginase| Allergic Disease                1
Blastic Plasmacytoid Dendritic Cell Neoplasm | Neoplasm                           1
B-cell acute leukemia                                                             1
accelerated phase Philadelphia chromosome positive myeloid leukemia               1
Inflammatory Myofibroblastic Tumor                                                1
Grade 2 astrocytoma                                                               1
Hodgkin’s disease                                                                 1
histiocytosis X                                              